# Deep learning example on image tiles

In [9]:
from anndata import AnnData
import torch.nn.functional as F
import numpy as np
from spatial_image import SpatialImage
import spatialdata as sd
from napari_spatialdata import Interactive
from spatialdata import SpatialData
from spatialdata.transformations import get_transformation
from spatialdata.dataloader.datasets import ImageTilesDataset
from tqdm import tqdm
from spatialdata import transform
from typing import Dict
import os
import scanpy as sc

from monai.networks.nets import DenseNet121
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import torchvision
import torch
from torch.utils.data import DataLoader
from pytorch_lightning import LightningDataModule
import torch.multiprocessing as mp


/Users/macbook/miniconda3/envs/ome/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/macbook/miniconda3/envs/ome/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/Users/macbook/embl/projects/basel/spatialdata/src/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


2023-04-10 16:09:12,099 - Created a temporary directory at /tmp/tmpx5zjosyv
2023-04-10 16:09:12,100 - Writing /tmp/tmpx5zjosyv/_remote_module_non_scriptable.py


## Preparing the data

In [12]:
print("current working directory:", os.getcwd())
SPATIALDATA_SANDBOX_PATH = "../../../spatialdata-sandbox"
GENERATED_DATA_PATH = os.path.join(
    SPATIALDATA_SANDBOX_PATH, "generated_data/xenium_visium_integration"
)
assert os.path.isdir(
    GENERATED_DATA_PATH
), f"{GENERATED_DATA_PATH} not found, please use symlinks to make it available"

XENIUM_SDATA_PATH = os.path.join(
    SPATIALDATA_SANDBOX_PATH, "xenium_rep1_io/data.zarr"
)
VISIUM_SDATA_PATH = os.path.join(
    SPATIALDATA_SANDBOX_PATH, "visium_associated_xenium_io/data.zarr"
)

assert os.path.isdir(XENIUM_SDATA_PATH)
assert os.path.isdir(VISIUM_SDATA_PATH)

xenium_sdata = sd.read_zarr(XENIUM_SDATA_PATH)
visium_sdata = sd.read_zarr(VISIUM_SDATA_PATH)

VISIUM_CLONAL_DATA = os.path.join(GENERATED_DATA_PATH, "visium_copyKat.h5ad")
assert os.path.isfile(VISIUM_CLONAL_DATA)
adata = sc.read_h5ad(VISIUM_CLONAL_DATA)
s = adata.obs["clone"]
s.index = visium_sdata.table.obs.index
visium_sdata.table.obs["clone"] = s

current working directory: /Users/macbook/embl/projects/basel/spatialdata-notebooks/notebooks/examples
2023-04-10 16:12:01,988 - root_attr: multiscales
2023-04-10 16:12:01,989 - datasets [{'coordinateTransformations': [{'scale': [1.0, 1.0, 1.0], 'type': 'scale'}], 'path': '0'}, {'coordinateTransformations': [{'scale': [1.0, 2.0, 2.0], 'type': 'scale'}], 'path': '1'}, {'coordinateTransformations': [{'scale': [1.0, 4.000310366232154, 4.0], 'type': 'scale'}], 'path': '2'}, {'coordinateTransformations': [{'scale': [1.0, 8.000620732464307, 8.0], 'type': 'scale'}], 'path': '3'}, {'coordinateTransformations': [{'scale': [1.0, 16.001241464928615, 16.003615002259377], 'type': 'scale'}], 'path': '4'}]
2023-04-10 16:12:01,993 - resolution: 0
2023-04-10 16:12:01,994 -  - shape ('c', 'y', 'x') = (1, 25778, 35416)
2023-04-10 16:12:01,994 -  - chunks =  ['1', '4096 (+ 1202)', '4096 (+ 2648)']
2023-04-10 16:12:01,995 -  - dtype = uint16
2023-04-10 16:12:01,997 - resolution: 1
2023-04-10 16:12:01,998 -

/Users/macbook/miniconda3/envs/ome/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/macbook/miniconda3/envs/ome/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
